## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Huicheng,CN,23.0385,116.2899,69.80,60,0,2.24,clear sky
1,1,Brae,GB,60.3964,-1.3530,44.60,87,100,12.66,overcast clouds
2,2,Urumqi,CN,43.8010,87.6005,60.80,39,0,11.18,clear sky
3,3,Vaini,TO,-21.2000,-175.2000,80.60,78,40,9.22,scattered clouds
4,4,Paamiut,GL,61.9940,-49.6678,27.43,75,100,12.84,light snow


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
prefered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
prefered_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
3,3,Vaini,TO,-21.2000,-175.2000,80.60,78,40,9.22,scattered clouds
5,5,Rikitea,PF,-23.1203,-134.9692,77.92,75,11,6.91,few clouds
7,7,Atuona,PF,-9.8000,-139.0333,80.28,79,18,18.79,few clouds
9,9,Walvis Bay,NaN,-22.9575,14.5053,86.00,41,25,8.99,scattered clouds
17,17,Saint-Philippe,RE,-21.3585,55.7679,77.00,83,90,11.50,overcast clouds


In [6]:
# 4a. Determine if there are any empty rows.
prefered_cities_df.count()

City_ID        275
City           275
Country        272
Lat            275
Lng            275
Max Temp       275
Humidity       275
Cloudiness     275
Wind Speed     275
Description    275
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
prefered_cities_df = prefered_cities_df.dropna(subset=['Country'])
prefered_cities_df.count()

City_ID        272
City           272
Country        272
Lat            272
Lng            272
Max Temp       272
Humidity       272
Cloudiness     272
Wind Speed     272
Description    272
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.

hotel_df = prefered_cities_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
3,Vaini,TO,80.60,scattered clouds,-21.2000,-175.2000,
5,Rikitea,PF,77.92,few clouds,-23.1203,-134.9692,
7,Atuona,PF,80.28,few clouds,-9.8000,-139.0333,
17,Saint-Philippe,RE,77.00,overcast clouds,-21.3585,55.7679,
19,Cabo Rojo,PR,81.00,scattered clouds,18.0866,-67.1457,
24,Arraial Do Cabo,BR,75.20,broken clouds,-22.9661,-42.0278,
43,Grand Gaube,MU,78.01,few clouds,-20.0064,57.6608,
46,Raga,SS,79.45,overcast clouds,8.4596,25.6780,
47,Kapaa,US,78.80,scattered clouds,22.0752,-159.3190,
48,Cabo San Lucas,MX,77.00,clear sky,22.8909,-109.9124,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {"radius": 5000, "type": "lodging", "key": g_key}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
# check for null
hotel_df.info()  # empty hotel names are empty strings, not null

<class 'pandas.core.frame.DataFrame'>
Int64Index: 272 entries, 3 to 708
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   City         272 non-null    object 
 1   Country      272 non-null    object 
 2   Max Temp     272 non-null    float64
 3   Description  272 non-null    object 
 4   Lat          272 non-null    float64
 5   Lng          272 non-null    float64
 6   Hotel Name   272 non-null    object 
dtypes: float64(3), object(4)
memory usage: 27.0+ KB


In [34]:
# 7. Drop the rows where there is no Hotel Name.  import numpy, search for empty string and replace with null.
import numpy as np
hotel_df["Hotel Name"] = hotel_df["Hotel Name"].replace([""],np.nan)
clean_hotel_df = hotel_df.dropna()  # Remove Null and set to new df.
clean_hotel_df.head(20)



,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
3,Vaini,TO,80.60,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
5,Rikitea,PF,77.92,few clouds,-23.1203,-134.9692,Pension Maro'i
7,Atuona,PF,80.28,few clouds,-9.8000,-139.0333,Villa Enata
17,Saint-Philippe,RE,77.00,overcast clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
19,Cabo Rojo,PR,81.00,scattered clouds,18.0866,-67.1457,Hotel Perichi's
24,Arraial Do Cabo,BR,75.20,broken clouds,-22.9661,-42.0278,Pousada Porto Praia
43,Grand Gaube,MU,78.01,few clouds,-20.0064,57.6608,Veranda Paul et Virginie Hotel & Spa
47,Kapaa,US,78.80,scattered clouds,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
48,Cabo San Lucas,MX,77.00,clear sky,22.8909,-109.9124,Hotel Tesoro Los Cabos
49,Meulaboh,ID,76.64,overcast clouds,4.1363,96.1285,Kost Pria


In [36]:
clean_hotel_df.info()  # check for null again

<class 'pandas.core.frame.DataFrame'>
Int64Index: 253 entries, 3 to 708
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   City         253 non-null    object 
 1   Country      253 non-null    object 
 2   Max Temp     253 non-null    float64
 3   Description  253 non-null    object 
 4   Lat          253 non-null    float64
 5   Lng          253 non-null    float64
 6   Hotel Name   253 non-null    object 
dtypes: float64(3), object(4)
memory usage: 15.8+ KB


In [37]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID") #also changes index column to City_ID instead of blank

In [51]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description} and {Max Temp}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [52]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))